# Yandex.Music

# Contents <a id='back'></a>

* [Introducción](#intro)
* [Descripción de los datos](#data_review)
    * [Conclusiones](#data_review_conclusions)
* [Data preprocessing](#data_preprocessing)
    * [3.1 Estilo del encabezado](#header_style)
    * [3.2 Valores ausentes](#missing_values)
    * [3.3 Duplicados](#duplicates)
    * [3.3 Duplicados implícitos](#duplicates_2)
    * [3.4 Conclusiones](#data_preprocessing_conclusions)
* [Prueba de hipótesis](#hypotheses)
    * [4.1 Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades](#activity)
    * [4.2 Hipótesis 2: música al principio y al final de la semana](#week)
    * [4.3 Hipótesis 3: preferencias de género en Springfield y Shelbyville](#genre)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>
El proyecto consiste en analizar un set de datos que contiene información sobre las preferencias musicales de los habitantes de dos ciudades: Springfield y Shelbyville. En base a esto, se pretende verificar cómo está la información en general, si hay que realizar una limpieza de datos, modificar categorías, comprobar datos ausentes o duplicados, entre otras acciones. Con esto, se espera verificar tres distintas hipótesis en base al comportamiento de las dos ciudades. 

### Objetivo: 
Verificar si son verdaderas o falsas las siguientes hipótesis: 
1. La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad. 
2. Los lunes por la mañana, los habitantes de Springfield y Shelbyville escuchan diferentes géneros. Lo mismo ocurre con los viernes por la noche. 
3. Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop mientras que en Shelbyville hay más aficionados al rap.

### Etapas 
Los datos del comportamiento del usuario se almacenan en el archivo `/datasets/music_project_en.csv`. No se tiene ninguna información sobre la calidad de los datos por lo que se examinará antes de probar las hipótesis. 

Primero, se evaluará la calidad de los datos y se verá si los problemas son significativos. En el preprocesamiento de datos, se tomará en cuenta los problemas más críticos.
 
El proyecto consistirá en tres etapas:
 1. Descripción de los datos
 2. Preprocesamiento de datos
 3. Prueba de hipótesis
 
[Volver a Contenidos](#back)

## Descripción de los datos <a id='data_review'></a>

A continuación se abrirán los datos en Yandex.Music para examinarlos.

Se importa la librería `pandas` y `numpy`.

In [1]:
# importando librerías
import pandas as pd
import numpy as np

Se lee el archivo `music_project_en.csv` de la carpeta `/datasets/` y se guarda en la variable `df`:

In [2]:
# leyendo el archivo y almacenándolo en df
df = pd.read_csv("/datasets/music_project_en.csv")

In [3]:
# resumen estadístico
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,21:51:22,Friday
freq,76,136,136,8850,45360,14,23149


En total tenemos 7 columnas, al parecer con un total de 65,079 filas. Arriba se puede observar la cantidad de datos únicos que tenemos en cada columna, donde al parecer tenemos varios usuarios que se repiten y comprobamos que solo tenemos información de dos ciudades. También verificamos qué dato se repite más en cada una de ellas. Al parecer tenemos más datos de la ciudad Springfield, de los días viernes y el género que más registros tiene es el pop. Por último tenemos la cantidad de veces que se repiten los datos que resaltan en cada columna. 

Obtenemos las 10 primeras filas de la tabla:

In [4]:
# obteniendo las 10 primeras filas de la tabla df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


A primera instancia se observa que los primeros 10 datos están correctos, no hay nada raro en ellos. 

Se obtenie la información general sobre la tabla:

In [5]:
# obteniendo información general sobre los datos en df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


La tabla contiene siete columnas como ya se comentó. Todas almacenan el mismo tipo de datos: objeto lo cual pueden ser manejable para todos los casos, incluso para `'time'`, aunque esto habrá que ver más adelante. 

De acuerdo con la documentación, se describe cada columna:
- `'userID'` — identificador del usuario
- `'Track'` — nombre de la canción
- `'artist'` — nombre del artista
- `'genre'` — género
- `'City'` — ciudad del usuario
- `'time'` — el periodo de tiempo exacto en que se reprodujo la pista
- `'Day'` — día de la semana

Se puede observar tres problemas con el estilo en los nombres de las columnas:
1. Algunos nombres están en mayúsculas, otros en minúsculas.
2. Hay algunos espacios en algunos nombres.
3. Hay valores nulos en las columnas `'Track', 'artist', 'genre'` ya que el número de valores de las columnas es diferente en estas en comparación con el resto. 

### Conclusiones <a id='data_review_conclusions'></a> 

Cada fila de la tabla almacena datos de la canción que fue reproducida. Algunas columnas describen la canción en sí: su título, el artista y el género. El resto transmite la información del usuario: la ciudad de la que viene, el tiempo que ha reproducido la canción. 

Está claro que los datos son suficientes para probar la hipótesis. Sin embargo, hay valores ausentes.

Para continuar, necesitamos preprocesar los datos.

[Volver a Contenidos](#back)

## Preprocesamiento de datos <a id='data_preprocessing'></a>
En esta sección procedemos a corregir el formato en los encabezados de las columnas y luego nos ocuparemos de los valores ausentes. Por último, comprobamos si hay duplicados en los datos.

### Estilo del encabezado <a id='header_style'></a>
Se imprime el encabezado de la columna:

In [6]:
# la lista de los nombres de las columnas en la tabla df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Se cambia los nombres de las columnas de acuerdo con las reglas del buen estilo:
* Si el nombre tiene varias palabras, se utiliza snake_case
* Todos los caracteres deben ser minúsculas
* Se elimina los espacios

In [7]:
# renombra las columnas
df = df.rename(
    columns={
        "  userID": "user_id",
        'Track': 'track',
        '  City  ': 'city',
        "Day" : "day"
    }
) 

Se comprueba el resultado imprimiendo los nombres de las columnas una vez más:

In [8]:
# comprobando el resultado: la lista de los nombres de las columnas
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Volver a Contenidos](#back)

### Valores ausentes <a id='missing_values'></a>
Primero verifiquemos en qué columnas hay valores ausentes en la tabla:

In [9]:
# verificación valores ausentes por columnas
df.columns[df.isnull().any()]

Index(['track', 'artist', 'genre'], dtype='object')

Ahora que comprobamos lo que vimos anteriormente, verifiquemos cuántos valores ausentes tenemos en total y en cada columna. 

In [10]:
# total de valores ausentes en todo el archivo
df.isna().sum().sum()

10108

In [11]:
# calculando valores ausentes por columnas
df.isna().sum()

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

Perfecto, pero comprobemos el porcentaje de valores ausentes que tenemos en cada columna, para verificar si es representativo o no los valores ausentes. 

In [12]:
# % de missing values por columna
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100, 2)))

user_id - 0.0%
track - 2.06%
artist - 11.63%
genre - 1.84%
city - 0.0%
time - 0.0%
day - 0.0%


La variable `'artist'` es la que más valores ausentes tiene y son más representativos, teniendo un total de 11.63% datos ausentes. En cambio para las columnas `'track'` y `'genre'`, es un porcentaje bastante bajo, no representativo.

Para poder manejar los datos de forma correcta, y no perder información importante en cuento a la cantidad de registros, se decide reemplazar todos los valores ausentes con la palabra `unknown`.

Antes de realizar esto, queremos verificar cuántas filas hay valores ausentes en las tres columnas (`'track', 'artist', 'genre'`). Para esto se realiza un filtro buscando valores ausentes para cada columna. 

In [13]:
# Mostrar las filas en las que existan valores nulos en las columnas "track", "genre", "artist" a la vez
missing_values = df.loc[(df["track"].isnull() == True) & (df["artist"].isnull() == True) & (df["genre"].isnull() == True)]
display(missing_values.head())
print(missing_values.shape)

,user_id,track,artist,genre,city,time,day
15,E3C5756F,NaN,NaN,NaN,Springfield,09:24:51,Monday
35,A8AE9169,NaN,NaN,NaN,Springfield,08:56:10,Monday
161,364C85C0,NaN,NaN,NaN,Springfield,20:06:58,Monday
210,C4990C90,NaN,NaN,NaN,Springfield,20:09:56,Monday
216,22B27E80,NaN,NaN,NaN,Springfield,13:34:16,Monday


(708, 7)


In [14]:
# número total de filas.
print(df.shape)

(65079, 7)


Procedemos con el reemplazo de los datos. 

In [15]:
# recorriendo los nombres de las columnas y reemplazando los valores ausentes con 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('Unknown')

Se vuelve a analizar los valores ausentes para verificar que no hay datos nulos.  

In [16]:
# % de missing values por columna
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100, 2)))

user_id - 0.0%
track - 0.0%
artist - 0.0%
genre - 0.0%
city - 0.0%
time - 0.0%
day - 0.0%


De igual forma, nos aseguramos de que la tabla no contiene más valores ausentes.

In [17]:
# contando valores ausentes
df.isna().sum().sum()

0

[Volver a Contenidos](#back)

### Duplicados <a id='duplicates'></a>
Verifiquemos si hay duplicados obvios en la tabla.

In [18]:
# contando duplicado obvios
df.duplicated().sum()

3826

Verificamos qué datos duplicados tenemos:

In [19]:
duplicated = df[df.duplicated()]
duplicated

,user_id,track,artist,genre,city,time,day
575,E7F07B46,Crazy,The Manhattans,rnb,Springfield,13:39:46,Monday
832,7671A47A,Johnny Go!,NikitA,ruspop,Springfield,21:59:33,Wednesday
1216,69467B01,Change It All,Harrison Storm,singer,Springfield,20:53:06,Wednesday
1754,13B1A573,Te Adoramos Jesús,Athenas,spiritual,Springfield,13:19:37,Monday
1964,B24668A0,Mad over You Mashup,Nana Fofie,singer,Springfield,20:36:51,Monday
...,...,...,...,...,...,...,...
65042,83E9C8C4,Buddhist Beat,Asian Zen Spa Music Meditation,ambient,Springfield,13:25:29,Monday
65056,2E25BCD2,Psychobitch,Easter,pop,Springfield,14:53:07,Friday
65059,F231C47E,All Summer in a Day,VHS Or BETA,electronic,Springfield,20:31:23,Friday
65067,F1B93F29,Poison Kiss,Centerstone,rock,Shelbyville,22:00:29,Monday


In [20]:
duplicated.sort_values(by = ["user_id", "track"]).head(10)

,user_id,track,artist,genre,city,time,day
39981,1001CE90,Invasion,Nick Arundel,videogame,Springfield,09:35:01,Monday
64923,100261F9,Jump,Kris Kross,hip,Springfield,14:42:21,Friday
18944,10189D10,Alia Aventuro,La Perdita Generacio,pop,Shelbyville,09:07:49,Wednesday
56707,10189D10,Jolene,Hey Ocean!,alternative,Shelbyville,09:42:17,Wednesday
39972,10189D10,Kontra krusadanto,Dochamar,pop,Shelbyville,14:34:24,Monday
47990,10189D10,Kontra krusadanto,Dochamar,pop,Shelbyville,14:34:24,Monday
13041,10270767,Hold On I'm Comin',Sam & Dave,rnb,Shelbyville,21:30:25,Wednesday
61664,103A549C,I Don't Belong To You,Milck,pop,Springfield,08:14:19,Friday
33986,1047FC7F,Benim Hikayem,Berkay,pop,Springfield,09:05:36,Friday
37156,104B7BA5,Mars,Fake Blood,dance,Shelbyville,21:11:42,Wednesday


Por ejemplo, en los índices 39972 y 47990 tenemos filas duplicadas, incluso más de dos veces, ya que se toma en cuenta que aquí solo aparecen las filas que se repiten más de una vez. El caso de filas duplicadas puede surgir a una mala importación de la base de datos inicial, a un registro duplicado al momento de guardar la información, entre otras razones. Sea cual sea, es importante eliminarlos, ya que podría generar ruido en nuestro análisis posterior. 

Para esto llamamos a un método en `pandas` para deshacerte de los duplicados obvios.

In [21]:
# eliminando duplicados obvios
df = df.drop_duplicates().reset_index(drop=True)

In [22]:
# verificación general de los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61253 entries, 0 to 61252
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  61253 non-null  object
 1   track    61253 non-null  object
 2   artist   61253 non-null  object
 3   genre    61253 non-null  object
 4   city     61253 non-null  object
 5   time     61253 non-null  object
 6   day      61253 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


Verificamos si hay duplicados obvios una vez más para asegurarte de que todos han sido eliminados:

In [23]:
# comprobando duplicados
df.duplicated().sum()

0

In [24]:
total = 65079 - 61253
total

3826

In [25]:
percent = total/65079 * 100
round(percent, 3)

5.879

Finalmente, nos hemos quedado con un total de 61,253 registros limpios, tomando en cuenta que en un inicio eran 65,079, eliminando un total de 3,826 datos, que representa el 5.879% de los datos iniciales que teníamos. Esta limpieza era necesaria ya que para el análisis posterior podría hacer ruido. 

### Duplicados implícitos <a id='duplicates_i'></a>

Hay que tomar en cuenta que nuestro análisis aún no ha terminado, primero se debe verificar qué datos tenemos en cada columna, para entender cómo están los datos en sí. 

In [26]:
# for loop para verificar la distribución de los datos en cada columna
columns = df.columns

for col in columns: 
    print(df[col].value_counts(normalize=True).head(10))
    print(f"Variable: {col}, # de valores únicos: {len(df[col].unique())}")
    print("")

A8AE9169    0.001159
7D166C63    0.000833
414F229D    0.000816
E8339398    0.000702
D6A03F1E    0.000588
EED05948    0.000588
B5496034    0.000555
B851E1A5    0.000539
3E4BFAA7    0.000506
39EDC574    0.000490
Name: user_id, dtype: float64
Variable: user_id, # de valores únicos: 41748

Unknown            0.020668
Brand              0.002073
So Long            0.001796
Going Back         0.001469
Moscow Calling     0.001420
All for You        0.001306
Balenciaga         0.001208
Bam Bag «Gucci»    0.001143
5 Min              0.001126
Perte de temps     0.001126
Name: track, dtype: float64
Variable: track, # de valores únicos: 39666

Unknown             0.115864
Kartvelli           0.002073
MALFA               0.001780
Real Bodrit         0.001469
The Seasons         0.001290
Irina Shok          0.001143
RELFY               0.001126
KoperniK            0.001126
Dr. Living Dead!    0.001110
Argishty (Duduk)    0.001012
Name: artist, dtype: float64
Variable: artist, # de valores únicos: 37

Podemos observar por lo menos que los 10 primeros datos de las 7 columnas se encuentran bien. Se puede ver que efectivamente hay datos solo de 2 ciudades, y de 3 días de la semana que son Lunes, Miércoles y Viernes.

Sin embargo, queremos verificar si no hay duplicados implicitos en las columnas `user_ID`, `artist`, `track` y `genre`. Para esto se convertirá todas en minúscula, y ver si los datos únicos se redujeron. Para esto se crea un nuevo dataframe. 

In [27]:
# copia del dataframe
df_copy = df.copy()

In [28]:
# convertir todos los datos en minúsculas
columns = df_copy.columns

for col in columns: 
    df_copy[col] = df_copy[col].str.lower()

In [29]:
# for loop para verificar la distribución de los datos en cada columna
columns = df_copy.columns

for col in columns: 
    print(df_copy[col].value_counts(normalize=True).head(10))
    print(f"Variable: {col}, # de valores únicos: {len(df_copy[col].unique())}")
    print("")

a8ae9169    0.001159
7d166c63    0.000833
414f229d    0.000816
e8339398    0.000702
d6a03f1e    0.000588
eed05948    0.000588
b5496034    0.000555
b851e1a5    0.000539
3e4bfaa7    0.000506
53b9a51d    0.000490
Name: user_id, dtype: float64
Variable: user_id, # de valores únicos: 41748

unknown            0.020668
brand              0.002073
so long            0.001796
going back         0.001469
moscow calling     0.001420
all for you        0.001371
balenciaga         0.001208
bam bag «gucci»    0.001143
perte de temps     0.001126
woman              0.001126
Name: track, dtype: float64
Variable: track, # de valores únicos: 39086

unknown             0.115864
kartvelli           0.002073
malfa               0.001780
real bodrit         0.001469
the seasons         0.001290
irina shok          0.001143
kopernik            0.001126
relfy               0.001126
dr. living dead!    0.001110
argishty (duduk)    0.001012
Name: artist, dtype: float64
Variable: artist, # de valores únicos: 37

Comparando los datos del dataframe `df` y `df_copy`, vemos que si se han reducido los datos en `track` y `artist`, donde la cantidad es mínima, pero ayudará para tener datos más limpios. 

Ahora vamos a deshacernos de los duplicados implícitos en la columna `genre`. Por ejemplo, el nombre de un género se puede escribir de varias formas. Dichos errores también pueden afectar a resultado.

Para analizar la columna, imprimimos una lista de nombres únicos de géneros, ordenados en orden alfabético.

In [30]:
# revisando en busca de duplicados implícitos
unique_genre = df_copy["genre"].unique()
unique_genre.sort()
print(unique_genre)
print(len(unique_genre))

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französisch' 

Hemos visto que el género hip-hop no está correcto porque tenemos palabras como hip, hop y hiphop que deberían estar corregidas, y así todas se cuenten por igual. Para este caso, creamos una función donde tenga dos argumentos, el primero la lista de palabras que están mal escritas y el segundo argumento la palabra con la que se va a cambiar. 

In [31]:
# función para reemplazar duplicados implícitos
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres: 
        df_copy['genre'] = df_copy['genre'].replace(wrong_genre, correct_genre)

In [32]:
# géneros hiphop
duplicates = ["hip" , "hiphop", "hop"]
genre = "hip-hop"
replace_wrong_genres(duplicates, genre)

Comparemos cómo están los datos antes y despúes:

In [33]:
#Se analiza la frecuencia de las variables.
print(df["genre"].value_counts().head(10))
print(len(df["genre"].value_counts()))
print()
print(df_copy["genre"].value_counts().head(10))
print(len(df_copy["genre"].value_counts()))

pop            8323
dance          6367
rock           5844
electronic     5522
hip            2975
classical      2262
alternative    2028
world          1947
ruspop         1910
rusrap         1725
Name: genre, dtype: int64
269

pop            8323
dance          6367
rock           5844
electronic     5522
hip-hop        3056
classical      2262
alternative    2028
world          1947
ruspop         1910
rusrap         1725
Name: genre, dtype: int64
266


Vemos antes que teníamos en el puesto 5to la palabra hip y ahora la palabra hip-hop pero con más cantidad de datos.  

Verifiquemos que nuestros datos estén correctos. 

In [34]:
# información general. 
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61253 entries, 0 to 61252
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  61253 non-null  object
 1   track    61253 non-null  object
 2   artist   61253 non-null  object
 3   genre    61253 non-null  object
 4   city     61253 non-null  object
 5   time     61253 non-null  object
 6   day      61253 non-null  object
dtypes: object(7)
memory usage: 3.3+ MB


Tenemos un total de 7 columnas y 61,253 filas. 

[Volver a Contenidos](#back)

### Conclusiones <a id='data_preprocessing_conclusions'></a>
Por el momento, podemos concluir los siguientes problemas encontrados:

- Estilos de encabezados incorrectos
- Valores ausentes
- Duplicados obvios e implícitos

Los encabezados han sido eliminados para conseguir que el procesamiento de la tabla sea más sencillo.

Reemplazamos todos los datos NaNs entre las tres columnas por la palabra `'Unknown'`. 

Se eliminó los duplicados obvios, y se limpió los duplicados implícitos en la columna `genre` del género hip-hop. La ausencia de duplicados implicitos hará que contabilicemos bien la cantidad de datos del género hip-hop.

Ahora ya podemos continuar probando las hipótesis. 

[Volver a Contenidos](#back)

## Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

De acuerdo con la primera hipótesis, los usuarios de Springfield y Shelbyville escuchan música de forma distinta. Se comprobará esto utilizando los datos de tres días de la semana: lunes, miércoles y viernes.

Para esto, vamos a realizar lo siguiente: 
* Dividir a los usuarios en grupos por ciudad.
* Comparar cuántas pistas reprodujo cada grupo el lunes, el miércoles y el viernes.


In [35]:
# contando las pistas reproducidas en cada ciudad
print(df_copy.groupby('city')[["track"]].count()) 

             track
city              
shelbyville  18512
springfield  42741


Springfield ha reproducido más pistas que Shelbyville. Pero eso no implica que los ciudadanos de Springfield escuchen música más a menudo. Esta ciudad es simplemente más grande y hay más usuarios.

Ahora agrupamos los datos por día de la semana y encontramos el número de pistas reproducidas del lunes, miércoles y viernes.

In [36]:
# calculando las pistas reproducidas en cada uno de los tres días
print(df_copy.groupby('day')[["track"]].count()) 

           track
day             
friday     21840
monday     21354
wednesday  18059


El miércoles fue el día más silencioso de todos. Pero podemos considerar las dos ciudades por separado para verificar si la conclusión es igual o diferente al resultado anterior. 

Para realizar esto más rápido vamos a crear la función `number_tracks()` para calcular el número de canciones reproducidas en un determinado día y ciudad. 

In [37]:
# función que calcula el número de canciones reproducidas en un determinado día y ciudad
def number_tracks(day, city):
    track_list = df_copy[(df_copy['day'] == day) & (df_copy['city'] == city)]
    track_list_count = track_list["user_id"].count()
    return track_list_count

En base a esta función, llamaremos seis veces modificando los parámetros:

In [38]:
# el número de canciones reproducidas en Springfield el lunes
m_sp = number_tracks("monday", "springfield")
print(m_sp)

15740


In [39]:
# el número de canciones reproducidas en Shelbyville el lunes
m_sh = number_tracks("monday", "shelbyville")
print(m_sh)

5614


In [40]:
# el número de canciones reproducidas en Springfield el miércoles
w_sp = number_tracks("wednesday", "springfield")
print(w_sp)

11056


In [41]:
# el número de canciones reproducidas en Shelbyville el miércoles
w_sh = number_tracks("wednesday", "shelbyville")
print(w_sh)

7003


In [42]:
# el número de canciones reproducidas en Springfield el viernes
f_sp = number_tracks("friday", "springfield")
print(f_sp)

15945


In [43]:
# el número de canciones reproducidas en Shelbyville el viernes
f_sh = number_tracks("friday", "shelbyville")
print(f_sh)

5895


Ahora vamos a llamar a un dataframe para crear una tabla con toda esta información, y poder resumirlo de mejor manera.

In [44]:
# variables para dataframe
columns = ['city','monday','wednesday','friday']
our_data = [['Shelbyville', m_sh, w_sh, f_sh], ['Springfield', m_sp, w_sp, f_sp]]

In [45]:
# tabla con los resultados
table = pd.DataFrame(data=our_data,columns=columns)
table

,city,monday,wednesday,friday
0,Shelbyville,5614,7003,5895
1,Springfield,15740,11056,15945


**Conclusiones**

Los datos revelan las diferencias en el comportamiento de los usuarios:

- En Springfield, el número de canciones reproducidas alcanzan el punto máximo los lunes y viernes mientras que los miércoles hay un descenso de la actividad.
- En Shelbyville, al contario, los usuarios escuchan más música los miércoles. La actividad de los usuarios los lunes y viernes es menor.

Así que la primera hipótesis parece ser correcta.

[Volver a Contenidos](#back)

### Hipótesis 2: música al principio y al final de la semana <a id='week'></a>

De acuerdo con la segunda hipótesis, los lunes por la mañana y los viernes por la noche los ciudadanos de Springfield escuchan géneros que difieren de aquellos que los usuarios de Shelbyville disfrutan.

Para esto vamos a definir dos variables, separando la tabla, una para cada ciudad. 

In [46]:
# obteniendo la tabla spr_general de las filas de df, 
# donde los valores en la columna 'city' es 'Springfield'
spr_general = df_copy[df_copy["city"] == "springfield"]

In [47]:
# obteniendo shel_general de las filas df,
# donde el valor de la columna 'city' es 'Shelbyville'
shel_general = df_copy[df_copy["city"] == "shelbyville"]

Con esto, podemos escribir una función genre_weekday() con cuatro parámetros:
* Una tabla para los datos (`df`)
* El día de la semana (`day`)
* La marca de fecha y hora en formato 'hh:mm' (`time1`)
* La marca de fecha y hora en formato 'hh:mm' (`time2`)

La función va a devolver información de los 15 géneros más populares de un día determinado en un período entre dos marcas de fecha y hora.

In [48]:
# función con los 15 génerps más populares
def genre_weekday(df, day, time1, time2):
    # día
    genre_df = df[df["day"] == day]
    # hora fin
    genre_df = genre_df[genre_df["time"] < time2]
    # hora inicio
    genre_df = genre_df[genre_df["time"] > time1]
    # se agrupa en base al género y cuenta la cantidad de usuarios
    genre_df_grouped = genre_df.groupby("genre")["user_id"].count()
    # se ordena de mayor a menor
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # devolvemos los primeros 15 géneros
    return genre_df_sorted[:15]

Con la función procedemos a comparar los resultados para Springfield y Shelbyville el lunes por la mañana (de 7:00 a 11:00) y el viernes por la tarde (de 17:00 a 23:00):

In [49]:
# llamando a la función para el lunes por la mañana en Springfield 
genre_weekday(spr_general, "monday", "07:00:00", "11:00:00")

genre
pop            781
dance          549
electronic     480
rock           474
hip-hop        286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: user_id, dtype: int64

In [50]:
# llamando a la función para el lunes por la mañana en Shelbyville (utilizando shel_general en vez de la tabla df)
genre_weekday(shel_general, "monday", "07:00:00", "11:00:00")

genre
pop            218
dance          182
rock           162
electronic     147
hip-hop         80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: user_id, dtype: int64

In [51]:
# llamando a la función para el viernes por la tarde en Springfield
genre_weekday(spr_general, "friday", "17:00:00", "21:00:00")

genre
pop            365
rock           287
dance          259
electronic     227
hip-hop        145
world          112
ruspop          90
alternative     82
rusrap          79
classical       75
unknown         68
soundtrack      50
jazz            48
rnb             47
folk            44
Name: user_id, dtype: int64

In [52]:
# llamando a la función para el viernes por la tarde en Shelbyville
genre_weekday(shel_general, "friday", "17:00:00", "21:00:00")

genre
rock           126
pop            120
dance          111
electronic      94
hip-hop         50
jazz            39
unknown         33
classical       32
alternative     32
world           29
ruspop          24
latin           22
rusrap          21
folk            21
metal           20
Name: user_id, dtype: int64

**Conclusión**

Habiendo comparado los 15 géneros más populares del lunes por la mañana podemos concluir lo siguiente:

1. Los usuarios de Springfield y Shelbyville escuchan música similar. Los cinco géneros más populares son los mismos, solo rock y electronica han intercambiado posiciones, al igual que otros géneros que son menos escuchados.

Para el viernes por la tarde, la situación es similar. Los géneros individuales varían algo pero, en general, los 15 más populares son parecidos en las dos ciudades. Aunque en Shelbyville en la tarde escucha más rock que pop. 

De esta forma, la segunda hipótesis ha sido parcialmente demostrada:
* Los usuarios escuchan música similar al principio y al final de la semana.
* No hay una gran diferencia entre Springfield y Shelbyville. En ambas ciudades, el pop es el género más popular, seguido del rock.

[Volver a Contenidos](#back)

### Hipótesis 3: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los ciudadanos de Springfield les gusta más el pop.

Agrupamos la tabla `spr_general` por género y encuentramos el número de canciones reproducidas de cada género con el método `count()`. Después, ordenamos el resultado en orden descendente, guardándolo en `spr_genres`.

In [53]:
# agrupación de la tabla spr_general por columna "genre" ordenado en orden descendente
spr_genres = spr_general.groupby("genre")["genre"].count()
spr_genres = spr_genres.sort_values(ascending=False)

In [54]:
# imprimiendo las 10 primeras filas de spr_genres
spr_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hip-hop        2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Realizamos lo mismo con los datos de Shelbyville con la tabla `shel_general`.

In [55]:
# agrupación de la tabla spr_general por columna "genre" ordenado en orden descendente
shel_genres = shel_general.groupby("genre")["genre"].count()
shel_genres = shel_genres.sort_values(ascending=False)

In [56]:
# imprimiendo las 10 primeras filas de shel_genres
shel_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hip-hop         960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusión**

La hipótesis ha sido rechazada:
* La música pop es el género más popular en Springfield, así como también en Shelbyville.
* El género dance está en segundo lugar como el género más escuchado.
* Por lo menos los primeros 5 géneros son los mismos en las dos ciudades, por lo que tienen los mismos gustos y comportamiento musical. 


[Volver a Contenidos](#back)

# Conclusiones <a id='end'></a>

Hemos probado las siguientes tres hipótesis:

1. La actividad de los usuarios difiere dependiendo del día de la semana y de las distintas ciudades. 
2. Los lunes por la mañana los residentes de Springfield y Shelbyville escuchan géneros similares al inicio, y distintos siendo los últimos géneros menos preferidos. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen preferencias similares. En ambas ciudades, Springfield y Shelbyville, se prefiere el pop.

Tras analizar los datos, concluimos:

1. La actividad del usuario en Springfield y Shelbyville depende del día de la semana aunque las ciudades varían de diferentes formas. 

La primera hipótesis ha sido aceptada completamente.

2. Las preferencias musicales no varían significativamente en el transcurso de la semana en Springfield y Shelbyville. Podemos observar pequeñas diferencias en el orden los lunes, pero:
* En Springfield y Shelbyville la gente lo que más escucha es la música pop.

Así que no podemos aceptar esta hipótesis, ya que las dos ciudades prefieren más la música pop, seguida de la música dance y rock. También debemos tener en cuenta que el resultado podría haber sido diferente si no fuera por los valores ausente.

3. Resulta que las preferencias musicales de los usuarios de Springfield y Shelbyville son bastante parecidas.

La tercera hipótesis es rechazada. Si hay alguna diferencia en las preferencias no se puede observar en los datos, ya que vimos que los gustos musicales eran los mismos y casi en el mismo orden todos, con pocas variaciones. 


[Volver a Contenidos](#back)